In this notebook, we resolve the problem
$$
\min_{g, u, p} \ \frac{1}{2}\int_{\Omega} \nabla u \cdot \nabla u~\textrm{d}x +  \frac{\alpha}{2} \int_{\partial \Omega_{\textrm{circle}}} g^2~\textrm{d}s
$$
where $\alpha>0$ and $(u,p)$ are the solution of Stokes equations
$$
\begin{split}-\nu \Delta u + \nabla p &= 0  \qquad \mathrm{in} \ \Omega \\
                  \mathrm{div }\  u &= 0  \qquad \mathrm{in} \ \Omega  \\\end{split}
$$
with boundary conditions given by
$$
\begin{split}
u &= g  \qquad \mathrm{on} \ \partial \Omega_{\textrm{cirlce}} \\
u &= u_D  \qquad \mathrm{on} \ \partial \Omega_{\textrm{in}} \\
u &= 0  \qquad \mathrm{on} \ \partial \Omega_{\textrm{walls}} \\
-\nu\dfrac{\partial u}{\partial n}+p n &= 0  \qquad \mathrm{on} \ \partial \Omega_{\textrm{out}} \\\end{split}
$$
where $u_0\in L^2 (\partial \Omega_{\textrm{in}})$ and $g\in L^2 (\partial \Omega_{\textrm{circle}})$

In [1]:
from __future__ import print_function
from dolfin import *
from dolfin_adjoint import *
import numpy as np
import scipy as sp
import cmath
import matplotlib.pyplot as plt
from mshr import *
import os

set_log_level(LogLevel.WARNING)
# Loading Mesh
mesh1 = Mesh("malla_real.xml")
boundaries1 = MeshFunction("size_t",mesh1,"malla_real_facet_region.xml")
# Defining FEM spaces
P2 = VectorElement("Lagrange",mesh1.ufl_cell(),2)
p1 = VectorElement("Lagrange",mesh1.ufl_cell(),1)
P1 = FiniteElement("Lagrange",mesh1.ufl_cell(),1)
TH = P2*P1
W = FunctionSpace(mesh1,TH)
V2 = FunctionSpace(mesh1,P2)
V = FunctionSpace(mesh1,P1)
v1 = FunctionSpace(mesh1,p1)

# Inflow
vel_max=1
uD=(Expression(('vel_max*(x[1]+1)*(1-x[1])','0'),degree=2,vel_max=vel_max))
# Control
g=Function(V2)
# Boundary Conditions
noslip=DirichletBC(W.sub(0),(0, 0),boundaries1,2)
inflow=DirichletBC(W.sub(0),uD,boundaries1,1)
circle=DirichletBC(W.sub(0),g,boundaries1,4)
bcs=[noslip,inflow,circle]
# Variational Problem
v, q = TestFunctions(W)
w = Function(W)
u, p = split(w)
nu=1
f=Constant((0,0))
F=(nu*inner(grad(u),grad(v))-div(v)*p+q*div(u))*dx
solve(F==0,w,bcs,solver_parameters={"newton_solver": {"maximum_iterations":15, "linear_solver":"mumps", "relative_tolerance": 1e-10, "absolute_tolerance": 1e-12}})
(u,p)=w.split()
m=Control(g)
# Defining functional
alpha=1e-6
J = assemble(1/2*(inner(grad(u),grad(u))*dx+alpha*inner(g,g)*ds(4)))
iter=0
s = Function(W,name="State")
vtkfile_control = File('Boundary/control.pvd')
vtkfile_state = File('Boundary/state.pvd')
# We can obtain extra information (plots of states and controls, values of J, etc.)  on each derivative evaluation
def eval_cb(j,dj,m):
    global iter
    global s
    uk, pk = split(s)
    v, q = TestFunctions(W)
    noslip_=DirichletBC(W.sub(0),(0, 0),boundaries1,2)
    inflow_=DirichletBC(W.sub(0),uD,boundaries1,1)
    circle_=DirichletBC(W.sub(0),m,boundaries1,4)
    bcs_=[noslip_,inflow_,circle_]
    F_=(nu*inner(grad(uk),grad(v))-div(v)*pk+q*div(uk))*dx
    solve(F_==0,s,bcs_,solver_parameters={"newton_solver": {"maximum_iterations":15, "linear_solver":"mumps", "relative_tolerance": 1e-10, "absolute_tolerance": 1e-12}})
    (uk,pk)=s.split()
    m.rename('control','control')
    uk.rename('u','u')
    vtkfile_control << (m,iter)
    vtkfile_state   << (uk,iter)
    if iter==0:
        print('{0}{1:^5}{0}{0}{2:^12}{0}'.format('|', 'Iter.', 'J(.)'))
    print('{0}{1:>5d}{0}{0}{2:>12.8f}{0}'.format('|',iter,j))
    iter=iter+1
# We define the minimize problem
RJ = ReducedFunctional(J,m,derivative_cb_post=eval_cb)
UI_OPT=minimize(RJ,method='L-BFGS-B',tol = 1e-8,options={'maxiter': 1000,'ftol': 1e-6, 'gtol': 1e-5})
# Plots of solutions were generated by eval_cb function

|Iter.||    J(.)    |
|    0|| 18.91127871|
|    1|| 14.38854649|
|    2||  2.52672879|
|    3||  5.18453727|
|    4||  2.38053833|
|    5||  2.32617707|
|    6||  2.29128671|
|    7||  2.25672401|
|    8||  2.26366542|
|    9||  2.24953347|
|   10||  2.24717345|
|   11||  2.24605999|
|   12||  2.24499267|
|   13||  2.24383669|
|   14||  2.24329263|
|   15||  2.24296433|
|   16||  2.24288501|
|   17||  2.24280499|
|   18||  2.24283933|
|   19||  2.24278843|
|   20||  2.24277533|
|   21||  2.24277307|
|   22||  2.24277191|


In [1]:
import os
os.system('python3 Boundary.py')

0